## 2. 开闭原则

开闭原则(Open-Closed Principle, OCP)：一个软件实体应当对扩展开放，对修改关闭。即软件实体应尽量在不修改原有代码的情况下进行扩展

## 2. 开闭原则

开闭原则(Open-Closed Principle, OCP)：一个软件实体应当对扩展开放，对修改关闭。即软件实体应尽量在不修改原有代码的情况下进行扩展

In [1]:
# 重构前, 假设需要添加一个 LineChart, 则需要直接修改原来的逻辑


class PieChart(object):
    """饼图"""
    def display(self):
        pass


class BarChart(object):
    """柱状图"""
    def display(self):
        pass


# 使用
type = 'xxx'
if type == 'pie':
    chart = PieChart()
    chart.display()
elif type == 'bar':
    chart = BarChart()
    chart.display()

In [ ]:
# 重构后, 可以结合工厂模式等进行横向扩展, 而不会影响原来的代码


class ChartDisplayer(object):
    chart = None
    
    def __init__(self, chart=None):
        self.chart = chart
    
    def set_chart(self, chart):
        self.chart = chart
    
    def display(self):
        if self.chart is None:
            raise
        return self.chart.display()


class Chart(object):
    def display(self):
        raise NotImplemented


class PieChart(Chart):
    def display(self): pass


class BarChart(Chart):
    def display(self): pass


class LineChart(Chart):
    def display(self): pass

## 3. 里氏置换原则

- [原文链接](http://blog.csdn.net/lovelion/article/details/7540445)

里氏代换原则(Liskov Substitution Principle, LSP)：所有引用基类（父类）的地方必须能透明地使用其子类的对象

在软件中将一个基类对象替换成它的子类对象，程序将不会产生任何错误和异常，反过来则不成立，如果一个软件实体使用的是一个子类对象的话，那么它不一定能够使用基类对象

在程序中尽量使用基类类型来对对象进行定义，而在**运行时再确定其子类类型**，用子类对象来替换父类对象

 里氏代换原则是实现开闭原则的重要方式之一

示例: 考虑一个给不同类型客户发电子邮件情况


In [ ]:
# 重构前, 考虑假设要添加一种新的用户类型SVIPUser


class EmailSender(object):
    def __init__(self, sender, title, content):
        self.sender = sender
        self.title = title
        self.content = content
    
    def send_to_normal_user(self, normal_user):
        return normal_user.send_email(self.sender, self.title, self.content)
    
    def send_to_vip_user(self, vip_user):
        return vip_user.sent_email(self.sender, self.title, self.content)


class NormalUser(object):
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def send_email(self, to, title, content):
        return 'Normal user sending email...'


class VIPUser(object):
    pass

In [ ]:
# 重构后


class EmailSender(object):
    def __init__(self, sender, title, content):
        self.sender = sender
        self.title = title
        self.content = content
    
    def send(self, user):
        return user.send_email(self.sender, self.title, self.content)


class User(object):
    def __init__(self, name, email):
        self.name = name
        self.email = email
    
    def send_email(self, to, title, content):
        raise NotImplemented


class NormalUser(User):
    def send_email(self, to, title, content):
        return 'Normal user sending email...'


class VIPUser(User):
    pass


class SVIPUser(User):
    pass

## 4. 依赖倒转原则

- [原文链接](http://blog.csdn.net/lovelion/article/details/7562783)

依赖倒转原则(Dependency Inversion  Principle, DIP)：抽象不应该依赖于细节，细节应当依赖于抽象。换言之，要针对接口编程，而不是针对实现编程

依赖倒转原则要求我们在程序代码中传递参数时或在关联关系中，尽量引用层次高的抽象层类，即使用接口和抽象类进行变量类型声明、参数类型声明、方法返回类型声明，以及数据类型的转换等，而不要用具体类来做这些事情

在实现依赖倒转原则时，我们需要针对抽象层编程，而将具体类的对象通过依赖注入(DependencyInjection, DI)的方式注入到其他对象中，依赖注入是指当一个对象要与其他对象发生依赖关系时，通过抽象来注入所依赖的对象。常用的注入方式有三种，分别是：构造注入，设值注入（Setter注入）和接口注入

开闭原则是目标，里氏代换原则是基础，依赖倒转原则是手段

## 5. 接口隔离原则

- [原文链接](http://blog.csdn.net/lovelion/article/details/7562842)

接口隔离原则(Interface  Segregation Principle, ISP)：使用多个专门的接口，而不使用单一的总接口，即客户端不应该依赖那些它不需要的接口

接口仅仅提供客户端需要的行为，客户端不需要的行为则隐藏起来，应当为客户端提供尽可能小的单独的接口，而不要提供大的总接口

## 6. 合成复用原则

合成复用原则(Composite Reuse Principle, CRP)：尽量使用对象组合，而不是继承来达到复用的目的

如果两个类之间是“Has-A”的关系应使用组合或聚合，如果是“Is-A”关系可使用继承


In [3]:
# 重构前, 假设要添加新的数据库链接方式, 如OracleDBUtil


class DBUtil(object):
    def get_connection(self):
        return 'Connection'


class UserDAO(DBUtil):
    def add_user(self, name):
        connection = self.get_connection()
        #...
        return 'User'


# 重构后, 使用依赖注入运行时确定使用哪种DBUtil


class DBUtil(object):
    def get_connection(self):
        return 'Connection'


class OracleDBUtil(DBUtil):
    def get_connection(self):
        return 'Oracle Connection'


class UserDAO(object):
    dbutil = None
    
    def set_dbutil(self, dbutil):
        self.dbutil = dbutil

    def add_user(self):
        connection = self.util.get_connection()
        # ..
        return 'User'

## 7. 迪米特法则

- [原文链接](http://blog.csdn.net/lovelion/article/details/7563445)

迪米特法则(Law of  Demeter, LoD)：一个软件实体应当尽可能少地与其他实体发生相互作用, 又称为最少知识原则(LeastKnowledge Principle, LKP)

在迪米特法则中，对于一个对象，其朋友包括以下几类：
	(1) 当前对象本身(self)
	(2) 以参数形式传入到当前对象方法中的对象
	(3) 当前对象的成员对象
	(4) 如果当前对象的成员对象是一个集合，那么集合中的元素也都是朋友
	(5) 当前对象所创建的对象

迪米特法则要求我们在设计系统时，应该尽量减少对象之间的交互，如果两个对象之间不必彼此直接通信，那么这两个对象就不应当发生任何直接的相互作用，如果其中的一个对象需要调用另一个对象的某一个方法的话，可以通过第三者转发这个调用。简言之，就是通过引入一个合理的第三者来降低现有对象之间的耦合度。

在将迪米特法则运用到系统设计中时，要注意下面的几点：在类的划分上，应当尽量创建松耦合的类，类之间的耦合度越低，就越有利于复用，一个处在松耦合中的类一旦被修改，不会对关联的类造成太大波及；在类的结构设计上，每一个类都应当尽量降低其成员变量和成员函数的访问权限；在类的设计上，只要有可能，一个类型应当设计成不变类；在对其他类的引用上，一个对象对其他对象的引用应当降到最低。